In [ ]:
import seagul.envs
import ray
import ray.rllib.agents.ppo as ppo
from ray.tune.logger import pretty_print
import tensorflow as tf
import time
from tqdm import trange

from ray.tune.registry import register_env
from seagul.envs.mujoco.five_link import FiveLinkWalkerEnv

env_name = "Walker2d-v2"

### Needed for custom envs only 
def env_creator(env_config):
    return FiveLinkWalkerEnv()  # return an env instance

register_env("five_link-v3", env_creator)
####


ray.init()
config = ppo.DEFAULT_CONFIG.copy()
config["num_gpus"] = 0
config["num_workers"] = 1
config["eager"] = False
config["model"]["fcnet_hiddens"] = []

trainer = ppo.PPOTrainer(config=config, env=env_name)

# Can optionally call trainer.restore(path) to load a checkpoint.

start = time.time()
for i in trange(500):
   # Perform one iteration of training the policy with ARS
    result = trainer.train()

checkpoint = trainer.save()
print("checkpoint saved at", checkpoint)
print(pretty_print(result))
ray.shutdown()

Couldn't import `requests` library. Be sure to install it on the client side.
Couldn't import `requests` library. Be sure to install it on the client side.
Traceback (most recent call last):
  File "/home/sgillen/miniconda3/envs/ray/lib/python3.6/site-packages/ray/tune/web_server.py", line 26, in <module>
    import requests  # `requests` is not part of stdlib.
  File "/home/sgillen/miniconda3/envs/ray/lib/python3.6/site-packages/requests/__init__.py", line 44, in <module>
    import chardet
ModuleNotFoundError: No module named 'chardet'
/home/sgillen/miniconda3/envs/ray/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sgillen/miniconda3/envs/ray/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or 

(pid=12798) 2019-11-08 17:51:21,116	WARNING policy_client.py:17 -- Couldn't import `requests` library. Be sure to install it on the client side.
(pid=12798) 2019-11-08 17:51:21,134	ERROR web_server.py:29 -- Couldn't import `requests` library. Be sure to install it on the client side.
(pid=12798) Traceback (most recent call last):
(pid=12798)   File "/home/sgillen/miniconda3/envs/ray/lib/python3.6/site-packages/ray/tune/web_server.py", line 26, in <module>
(pid=12798)     import requests  # `requests` is not part of stdlib.
(pid=12798)   File "/home/sgillen/miniconda3/envs/ray/lib/python3.6/site-packages/requests/__init__.py", line 44, in <module>
(pid=12798)     import chardet
(pid=12798) ModuleNotFoundError: No module named 'chardet'
(pid=12798) /home/sgillen/miniconda3/envs/ray/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (ty

  0%|          | 0/500 [00:00<?, ?it/s]

(pid=12798) /home/sgillen/miniconda3/envs/ray/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
(pid=12798)   "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
(pid=12798) 2019-11-08 17:51:23,202	INFO rollout_worker.py:467 -- Generating sample batch of size 200
(pid=12798) 2019-11-08 17:51:23,203	INFO sampler.py:310 -- Raw obs from env: { 0: { 'agent0': np.ndarray((17,), dtype=float64, min=-0.005, max=1.251, mean=0.074)}}
(pid=12798) 2019-11-08 17:51:23,203	INFO sampler.py:311 -- Info return from env: {0: {'agent0': None}}
(pid=12798) 2019-11-08 17:51:23,203	INFO sampler.py:409 -- Preprocessed obs: np.ndarray((17,), dtype=float64, min=-0.005, max=1.251, mean=0.074)
(pid=12798) 2019-11-08 17:51:23,203	INFO sampler.py:413 -- Filtered obs: np.ndarray((17,), dtype=float64, min=-0.005, max=1.251, mean=0.074)
(pid=12798) 2019-

2019-11-08 17:51:27,600	INFO tf_policy.py:358 -- Optimizing variable <tf.Variable 'default_policy/fc_out/kernel:0' shape=(17, 12) dtype=float32>
2019-11-08 17:51:27,601	INFO tf_policy.py:358 -- Optimizing variable <tf.Variable 'default_policy/fc_out/bias:0' shape=(12,) dtype=float32>
2019-11-08 17:51:27,602	INFO tf_policy.py:358 -- Optimizing variable <tf.Variable 'default_policy/value_out/kernel:0' shape=(17, 1) dtype=float32>
2019-11-08 17:51:27,602	INFO tf_policy.py:358 -- Optimizing variable <tf.Variable 'default_policy/value_out/bias:0' shape=(1,) dtype=float32>
2019-11-08 17:51:27,607	INFO multi_gpu_impl.py:146 -- Training on concatenated sample batches:

{ 'inputs': [ np.ndarray((4000, 6), dtype=float32, min=-4.13, max=4.347, mean=0.005),
              np.ndarray((4000,), dtype=float32, min=-2.091, max=2.194, mean=-0.034),
              np.ndarray((4000, 17), dtype=float32, min=-10.0, max=10.0, mean=-1.021),
              np.ndarray((4000,), dtype=float32, min=-21.336, max=-5.62

In [2]:
import gym
import numpy as np
import matplotlib.pyplot as plt

env = gym.make(env_name)
obs = env.reset()

action_hist = []
state_hist  = []
reward_hist = []

done = False
while not done:
    actions = trainer.compute_action(obs.flatten())
    obs, reward, done, _ = env.step(np.asarray(actions))
    action_hist.append(np.copy(np.clip(actions, -10, 10)))
    state_hist.append(np.copy(obs))
    reward_hist.append(np.copy(reward))
    if done:
        break


print(sum(reward_hist))
plt.plot(action_hist)
plt.figure()
plt.plot(state_hist)

2019-11-05 14:32:29,688	INFO tf_run_builder.py:92 -- Executing TF run without tracing. To dump TF timeline traces to disk, set the TF_TIMELINE_DIR environment variable.


319.22329804052384
